In [2]:
import os
from tqdm import tqdm
import pandas as pd
import sys
import time
import logging
import numpy as np
import matplotlib.pyplot as plt
import cv2
import re
from sklearn.neighbors import NearestNeighbors
from pathlib import Path 

In [3]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

In [29]:
def nearest_neighbour(new_skeleton, various_skeletons):
    joint=1
    neigh.fit([[new_skeleton1[-1][0][joint],  new_skeleton1[-1][1][joint]]])
    if len(various_skeletons) ==1:
        dist, nn = neigh.kneighbors([[list(various_skeletons.values())[0][0][joint], list(various_skeletons.values())[0][1][joint]]], 
                                        return_distance = True)
        return([dist])
    elif len(various_skeletons)==2:
        dist1, nn = neigh.kneighbors([[list(various_skeletons.values())[0][0][joint], list(various_skeletons.values())[0][1][joint]]], 
                                        return_distance = True)
        dist2, nn = neigh.kneighbors([[list(various_skeletons.values())[1][0][joint], list(various_skeletons.values())[1][1][joint]]], 
                                        return_distance = True)
        return([dist1, dist2])
    elif len(various_skeletons) ==3:
        dist1, nn = neigh.kneighbors([[list(various_skeletons.values())[0][0][joint], list(various_skeletons.values())[0][1][joint]]], 
                                        return_distance = True)
        dist2, nn = neigh.kneighbors([[list(various_skeletons.values())[1][0][joint], list(various_skeletons.values())[1][1][joint]]], 
                                        return_distance = True)
        dist3, nn = neigh.kneighbors([[list(various_skeletons.values())[2][0][joint], list(various_skeletons.values())[2][1][joint]]], 
                                        return_distance = True)
        return([dist1,dist2,dist3])

In [26]:
skeletons[20]

{'skeleton1': [array([476.16, 499.2 , 453.12, 407.04, 422.4 , 545.28, 545.28, 476.16,
         437.76,   0.  ,   0.  , 514.56, 460.8 ,   0.  , 476.16, 491.52,
           0.  , 522.24]),
  array([132.48, 155.52, 144.  , 230.4 , 282.24, 172.8 , 264.96, 299.52,
         288.  ,   0.  ,   0.  , 299.52, 374.4 ,   0.  , 120.96, 126.72,
           0.  , 126.72]),
  array([0.31, 0.68, 0.64, 0.72, 0.37, 0.66, 0.71, 0.54, 0.17, 0.  , 0.  ,
         0.32, 0.19, 0.  , 0.22, 0.37, 0.  , 0.6 ])],
 'skeleton2': [array([391.68, 445.44,   0.  ,   0.  ,   0.  , 445.44, 391.68, 322.56,
           0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  , 407.04,
           0.  , 437.76]),
  array([357.12, 362.88,   0.  ,   0.  ,   0.  , 385.92, 432.  , 420.48,
           0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  , 339.84,
           0.  , 339.84]),
  array([0.15, 0.15, 0.  , 0.  , 0.  , 0.35, 0.66, 0.62, 0.  , 0.  , 0.  ,
         0.  , 0.  , 0.  , 0.  , 0.18, 0.  , 0.51])]}

In [28]:
list(skeletons[20].values())[0][0][1]

499.2

In [22]:
neigh.fit([[514,155]])
dist, nn = neigh.kneighbors([[506,167]], return_distance = True)
print(dist)

[[14.4222051]]


In [36]:
def check_missing(skeletons):
    """if value is missing then it returns True"""
    temp_list = []
    for key,value in skeletons.items():
        
        if value[0][1] ==0:
            temp_list.append(True)
        else:
            temp_list.append(False)
    return(np.all(temp_list))

In [4]:
openpose_path = '/Users/andreibirladeanu/Documents/Data/meal_openpose/1053_meal/'

In [5]:
coords = [file for file in sorted_alphanumeric(os.listdir(openpose_path)) if file[0] !="."]


In [44]:
thr = 100
skeletons = []
frames = []
for frame in coords:
    data = pd.read_csv(openpose_path + frame)
    data = data.fillna(0)
    if (len(data.columns)-1)/3 == 1:
        skeletons.append({'skeleton1':[np.array(data['x_1']), np.array(data['y_1']), np.array(data['prob_1'])]})
        frames.append(int(str(frame.split('.')[0]).split("_")[1]))
    elif (len(data.columns)-1)/3 == 2:
        skeletons.append({'skeleton1':[np.array(data['x_1']), np.array(data['y_1']),  np.array(data['prob_1'])],'skeleton2':[np.array(data['x_2']), np.array(data['y_2']),
                                                                                                                         np.array(data['prob_2'])]})
        frames.append(int(str(frame.split('.')[0]).split("_")[1]))
    elif (len(data.columns)-1)/3 == 3:
        skeletons.append({'skeleton1':[np.array(data['x_1']), np.array(data['y_1']),  np.array(data['prob_1'])],'skeleton2':[np.array(data['x_2']), np.array(data['y_2']),
                         np.array(data['prob_2'])],
                      'skeleton3':[np.array(data['x_3']), np.array(data['y_3']),  np.array(data['prob_3'])]}) 
        frames.append(int(str(frame.split('.')[0]).split("_")[1]))

In [104]:
print(skeletons[0])

{'skeleton1': [array([506.88, 522.24, 476.16, 414.72, 414.72, 568.32, 552.96, 468.48,
       445.44,   0.  ,   0.  , 522.24,   0.  ,   0.  , 499.2 , 522.24,
         0.  , 552.96]), array([109.44, 144.  , 120.96, 167.04, 224.64, 161.28, 253.44, 293.76,
       253.44,   0.  ,   0.  , 282.24,   0.  ,   0.  ,  97.92, 103.68,
         0.  , 103.68]), array([0.65, 0.71, 0.6 , 0.66, 0.41, 0.67, 0.74, 0.23, 0.2 , 0.  , 0.  ,
       0.26, 0.  , 0.  , 0.44, 0.61, 0.  , 0.79])]}


In [45]:
### extracting skel 1
joint = 1
thr = 100
neigh = NearestNeighbors(n_neighbors=1, algorithm = 'brute', metric='euclidean')
new_skeleton1 = []
main_kpt = 1
fallback = 0
frame_no = []
for i, frm in zip(range(len(skeletons)), frames): 
    if check_missing(skeletons[i])==True: ### checks if the joint was detected in all the skeletons
        continue
    elif new_skeleton1 == []:
        new_skeleton1.append(skeletons[i]['skeleton1'])
        skeletons[i].pop('skeleton1')
        frame_no.append(frm)
    elif len(skeletons[i])==1:
        dist = nearest_neighbour(new_skeleton1[-1], skeletons[i])
        #print(dist[0][0][0], frm)
        if dist[0][0][0] < thr:
            new_skeleton1.append(list(skeletons[i].values())[0])
            frame_no.append(frm)
            skeletons[i].pop(list(skeletons[i].keys())[0])
        else:
            continue                  
    elif len(skeletons[i])==2:
        dist = nearest_neighbour(new_skeleton1[-1], skeletons[i])
        distance_1 = dist[0][0][0]; distance_2 = dist[1][0][0]
        if distance_1 < distance_2 and distance_1< thr:
            new_skeleton1.append(skeletons[i]['skeleton1'])
            frame_no.append(frm)
            skeletons[i].pop('skeleton1') 
        elif distance_1 > distance_2 and distance_2<thr:
            new_skeleton1.append(skeletons[i]['skeleton2'])
            frame_no.append(frm)
            skeletons[i].pop('skeleton2')  
        else:
            continue
    elif len(skeletons[i])==3:
        dist = nearest_neighbour(new_skeleton1[-1], skeletons[i])
        distance_1 = dist[0][0][0]; distance_2 = dist[1][0][0]; distance_3 = dist[2][0][0]
        if distance_1 < distance_2 < distance_3 and distance_1 < thr:
            new_skeleton1.append(skeletons[i]['skeleton1'])
            frame_no.append(frm)
            skeletons[i].pop('skeleton1')  
        elif distance_2 < distance_1 < distance_3 and distance_2 < thr:
            new_skeleton1.append(skeletons[i]['skeleton2'])
            frame_no.append(frm)
            skeletons[i].pop('skeleton2')
        elif distance_3 < distance_1 < distance_2 and distance_3 < thr:
            new_skeleton1.append(skeletons[i]['skeleton3'])
            frame_no.append(frm)
            skeletons[i].pop('skeleton3')
        else:
            continue
for skeleton, f_no in zip(new_skeleton1, frame_no):
    out = pd.DataFrame(np.transpose(skeleton),  columns=['x', 'y', 'p'])
    filepath = Path('/Users/andreibirladeanu/Documents/Data/1053_meal_openpose/' +'skel1/' +str(f_no) +'.csv')
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    out.to_csv(filepath)

print('done')
    

done


In [181]:
list(skeletons[20].keys())[0]


'skeleton2'

In [46]:
### second pass skeleton2:
neigh = NearestNeighbors(n_neighbors=1, algorithm = 'brute', metric='euclidean')
new_skeleton2 = []
frame_no = []
for i, frm in zip(range(len(skeletons)), frames): 
    if skeletons[i] == {}:
        continue
    elif check_missing(skeletons[i])==True: ### checks if the joint was detected in all the skeletons
        continue
    elif new_skeleton2 == []:
        new_skeleton2.append(list(skeletons[i].values())[0])
        skeletons[i].pop(list(skeletons[i].keys())[0])
        frame_no.append(frm)
    elif len(skeletons[i])==1:
        dist = nearest_neighbour(new_skeleton1[-1], skeletons[i])
        distance = dist[0][0][0]
        if distance < thr:
            new_skeleton2.append(list(skeletons[i].values())[0])
            frame_no.append(frm)
            skeletons[i].pop(list(skeletons[i].keys())[0])
        else:
            continue                  
    elif len(skeletons[i]) ==2:
        dist = nearest_neighbour(new_skeleton1[-1], skeletons[i])
        distance_1 = dist[0][0][0]; distance_2 = dist[1][0][0]
        if distance_1 < distance_2 and distance_1< thr:
            new_skeleton2.append(list(skeletons[i].values())[0])
            frame_no.append(frm)
            skeletons[i].pop(list(skeletons[i].keys())[0])
        elif distance_1 > distance_2 and distance_2<thr:
            new_skeleton2.append(list(skeletons[i].values())[1])
            frame_no.append(frm)
            skeletons[i].pop(list(skeletons[i].keys())[1])
        else:
            continue
for skeleton, f_no in zip(new_skeleton2, frame_no):
    out = pd.DataFrame(np.transpose(skeleton),  columns=['x', 'y', 'p'])
    filepath = Path('/Users/andreibirladeanu/Documents/Data/1053_meal_openpose/' +'skel2/' +str(f_no) +'.csv')
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    out.to_csv(filepath)

In [47]:
#### third pass
new_skeleton3 = []
frame_no = []
for i, frm in zip(range(len(skeletons)), frames): 
    if skeletons[i] == {}:
        continue
    elif check_missing(skeletons[i])==True: ### checks if the joint was detected in all the skeletons
        continue
    if len(skeletons[i]) != {}:
        new_skeleton3.append(list(skeletons[i].items())[0][1])
        frame_no.append(frm)
for skeleton, f_no in zip(new_skeleton3, frame_no):
    out = pd.DataFrame(np.transpose(skeleton),  columns=['x', 'y', 'p'])
    filepath = Path('/Users/andreibirladeanu/Documents/Data/1053_meal_openpose/' +'skel3/' +str(f_no) +'.csv')
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    out.to_csv(filepath)


In [ ]:
print('a')

In [119]:
skeletons[1]['skeleton1'][1]

array([109.44, 144.  , 120.96, 172.8 , 236.16, 161.28, 259.2 , 288.  ,
       259.2 ,   0.  ,   0.  , 276.48,   0.  ,   0.  , 103.68, 103.68,
         0.  , 109.44])

In [73]:
check_missing(skeletons[20])

False

In [72]:
check_missing(skeletons[20]) == False

True

In [54]:
for key,value in skeletons[20].items():
    if value[0][0] !=0:
        print (False)
    else:
        print(True)

False
False


In [ ]:
dist, nn =  neigh.kneighbors(skeletons[20].items()

In [39]:
skeletons[20]

{'skeleton1': [array([476.16, 499.2 , 453.12, 407.04, 422.4 , 545.28, 545.28, 476.16,
         437.76,   0.  ,   0.  , 514.56, 460.8 ,   0.  , 476.16, 491.52,
           0.  , 522.24]),
  array([132.48, 155.52, 144.  , 230.4 , 282.24, 172.8 , 264.96, 299.52,
         288.  ,   0.  ,   0.  , 299.52, 374.4 ,   0.  , 120.96, 126.72,
           0.  , 126.72]),
  array([0.31, 0.68, 0.64, 0.72, 0.37, 0.66, 0.71, 0.54, 0.17, 0.  , 0.  ,
         0.32, 0.19, 0.  , 0.22, 0.37, 0.  , 0.6 ])],
 'skeleton2': [array([391.68, 445.44,   0.  ,   0.  ,   0.  , 445.44, 391.68, 322.56,
           0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  , 407.04,
           0.  , 437.76]),
  array([357.12, 362.88,   0.  ,   0.  ,   0.  , 385.92, 432.  , 420.48,
           0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  , 339.84,
           0.  , 339.84]),
  array([0.15, 0.15, 0.  , 0.  , 0.  , 0.35, 0.66, 0.62, 0.  , 0.  , 0.  ,
         0.  , 0.  , 0.  , 0.  , 0.18, 0.  , 0.51])]}